In [49]:
pip install openmeteo-requests

In [50]:
pip install requests-cache retry-requests numpy pandas

In [51]:
import openmeteo_requests

import pandas as pd
import requests_cache
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
	"latitude": 40.38,
	"longitude": 49.87,
	"start_date": "2025-11-24",
	"end_date": "2025-12-25",
	"daily": ["apparent_temperature_mean", "apparent_temperature_max", "apparent_temperature_min", "weather_code", "precipitation_sum", "rain_sum", "snowfall_sum", "precipitation_hours", "sunrise", "sunset", "daylight_duration", "sunshine_duration"],
	"timezone": "GMT",
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates: {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation: {response.Elevation()} m asl")
print(f"Timezone: {response.Timezone()}{response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0: {response.UtcOffsetSeconds()}s")

# Process daily data. The order of variables needs to be the same as requested.
daily = response.Daily()
daily_apparent_temperature_mean = daily.Variables(0).ValuesAsNumpy()
daily_apparent_temperature_max = daily.Variables(1).ValuesAsNumpy()
daily_apparent_temperature_min = daily.Variables(2).ValuesAsNumpy()
daily_weather_code = daily.Variables(3).ValuesAsNumpy()
daily_precipitation_sum = daily.Variables(4).ValuesAsNumpy()
daily_rain_sum = daily.Variables(5).ValuesAsNumpy()
daily_snowfall_sum = daily.Variables(6).ValuesAsNumpy()
daily_precipitation_hours = daily.Variables(7).ValuesAsNumpy()
daily_sunrise = daily.Variables(8).ValuesInt64AsNumpy()
daily_sunset = daily.Variables(9).ValuesInt64AsNumpy()
daily_daylight_duration = daily.Variables(10).ValuesAsNumpy()
daily_sunshine_duration = daily.Variables(11).ValuesAsNumpy()

daily_data = {"date": pd.date_range(
	start = pd.to_datetime(daily.Time(), unit = "s", utc = True),
	end =  pd.to_datetime(daily.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = daily.Interval()),
	inclusive = "left"
)}

daily_data["apparent_temperature_mean"] = daily_apparent_temperature_mean
daily_data["apparent_temperature_max"] = daily_apparent_temperature_max
daily_data["apparent_temperature_min"] = daily_apparent_temperature_min
daily_data["weather_code"] = daily_weather_code
daily_data["precipitation_sum"] = daily_precipitation_sum
daily_data["rain_sum"] = daily_rain_sum
daily_data["snowfall_sum"] = daily_snowfall_sum
daily_data["precipitation_hours"] = daily_precipitation_hours
daily_data["sunrise"] = daily_sunrise
daily_data["sunset"] = daily_sunset
daily_data["daylight_duration"] = daily_daylight_duration
daily_data["sunshine_duration"] = daily_sunshine_duration

daily_dataframe = pd.DataFrame(data = daily_data)
print("\nDaily data\n", daily_dataframe)


Coordinates: 40.38664245605469°N 49.816898345947266°E
Elevation: -30.0 m asl
Timezone: NoneNone
Timezone difference to GMT+0: 0s

Daily data
                         date  apparent_temperature_mean  \
0  2025-11-24 00:00:00+00:00                  10.566236   
1  2025-11-25 00:00:00+00:00                   9.047736   
2  2025-11-26 00:00:00+00:00                  10.349210   
3  2025-11-27 00:00:00+00:00                   9.747184   
4  2025-11-28 00:00:00+00:00                  10.684850   
5  2025-11-29 00:00:00+00:00                  12.375562   
6  2025-11-30 00:00:00+00:00                  12.136742   
7  2025-12-01 00:00:00+00:00                  12.838340   
8  2025-12-02 00:00:00+00:00                   9.224884   
9  2025-12-03 00:00:00+00:00                   6.515053   
10 2025-12-04 00:00:00+00:00                   8.614886   
11 2025-12-05 00:00:00+00:00                   9.967941   
12 2025-12-06 00:00:00+00:00                   8.611236   
13 2025-12-07 00:00:00+00:00    

In [52]:
daily_dataframe.sample(7)

,date,apparent_temperature_mean,apparent_temperature_max,apparent_temperature_min,weather_code,precipitation_sum,rain_sum,snowfall_sum,precipitation_hours,sunrise,sunset,daylight_duration,sunshine_duration
30,2025-12-24 00:00:00+00:00,5.456863,8.938315,1.429484,51.0,0.3,0.3,0.0,3.0,1766548896,1766582345,33449.636719,13713.772461
19,2025-12-13 00:00:00+00:00,5.365142,9.258038,-1.995795,55.0,2.1,2.1,0.0,5.0,1765598114,1765631676,33558.101562,24202.433594
6,2025-11-30 00:00:00+00:00,12.136742,13.390526,10.803409,51.0,0.1,0.1,0.0,1.0,1764474213,1764508519,34295.386719,22695.160156
3,2025-11-27 00:00:00+00:00,9.747184,11.085346,8.796183,51.0,0.1,0.1,0.0,1.0,1764214823,1764249380,34545.281250,17480.500000
28,2025-12-22 00:00:00+00:00,1.857808,4.192529,-0.751052,3.0,0.0,0.0,0.0,0.0,1766376042,1766409476,33434.660156,26097.347656
2,2025-11-26 00:00:00+00:00,10.349210,12.784582,9.135027,3.0,0.0,0.0,0.0,0.0,1764128359,1764163005,34633.781250,23677.460938
17,2025-12-11 00:00:00+00:00,7.607927,8.521471,6.519595,51.0,0.4,0.4,0.0,4.0,1765425225,1765458859,33628.488281,16633.044922


In [53]:
daily_dataframe

,date,apparent_temperature_mean,apparent_temperature_max,apparent_temperature_min,weather_code,precipitation_sum,rain_sum,snowfall_sum,precipitation_hours,sunrise,sunset,daylight_duration,sunshine_duration
0,2025-11-24 00:00:00+00:00,10.566236,11.816883,9.033800,3.0,0.000000,0.000000,0.0,0.0,1763955430,1763990261,34817.875000,29395.365234
1,2025-11-25 00:00:00+00:00,9.047736,11.811586,7.107851,3.0,0.000000,0.000000,0.0,0.0,1764041894,1764076632,34724.691406,31121.869141
2,2025-11-26 00:00:00+00:00,10.349210,12.784582,9.135027,3.0,0.000000,0.000000,0.0,0.0,1764128359,1764163005,34633.781250,23677.460938
3,2025-11-27 00:00:00+00:00,9.747184,11.085346,8.796183,51.0,0.100000,0.100000,0.0,1.0,1764214823,1764249380,34545.281250,17480.500000
4,2025-11-28 00:00:00+00:00,10.684850,13.003252,8.378041,3.0,0.000000,0.000000,0.0,0.0,1764301287,1764335758,34459.292969,26604.222656
5,2025-11-29 00:00:00+00:00,12.375562,14.211342,10.481182,53.0,3.700000,3.700000,0.0,15.0,1764387750,1764422137,34375.953125,1319.857666
6,2025-11-30 00:00:00+00:00,12.136742,13.390526,10.803409,51.0,0.100000,0.100000,0.0,1.0,1764474213,1764508519,34295.386719,22695.160156
7,2025-12-01 00:00:00+00:00,12.838340,13.896776,11.037653,51.0,1.800000,1.800000,0.0,9.0,1764560675,1764594903,34217.710938,18366.601562
8,2025-12-02 00:00:00+00:00,9.224884,11.617625,6.936610,51.0,0.500000,0.500000,0.0,4.0,1764647136,1764681289,34143.066406,26006.986328
9,2025-12-03 00:00:00+00:00,6.515053,7.210442,5.860048,51.0,0.300000,0.300000,0.0,2.0,1764733596,1764767677,34071.570312,12437.929688


In [54]:
import requests

countries = [
    "Albania", "Algeria", "Angola", "Argentina", "Armenia", "Australia", "Austria",
    "Azerbaijan", "Bahamas", "Bahrain", "Bangladesh", "Belarus", "Belgium", "Botswana",
    "Brazil", "Bulgaria", "Burkina Faso", "Burundi", "Cameroon", "Canada", "Central African Republic",
    "Chile", "Colombia", "Croatia", "Denmark", "Dominican Republic", "Ecuador", "Egypt",
    "El Salvador", "Eritrea", "Estonia", "Finland", "France", "Germany", "Ghana",
    "Greece", "Guatemala", "Guinea", "Guyana", "Haiti", "Honduras", "Hungary",
    "India", "Indonesia", "Iraq", "Ireland", "Italy", "Jamaica", "Japan",
    "Kazakhstan", "Kenya", "Latvia", "Lebanon", "Lesotho", "Libya", "Madagascar",
    "Malawi", "Malaysia", "Mali", "Mauritania", "Mauritius", "Mexico", "Montenegro",
    "Morocco", "Mozambique", "Namibia", "Nepal", "Netherlands", "New Zealand", "Nicaragua",
    "Niger", "Norway", "Pakistan", "Papua New Guinea", "Peru", "Poland", "Portugal",
    "Qatar", "Romania", "Rwanda", "Saudi Arabia", "Senegal", "Slovenia", "South Africa",
    "Spain", "Sri Lanka", "Sudan", "Suriname", "Sweden", "Switzerland", "Tajikistan",
    "Thailand", "Tunisia", "Turkey", "Uganda", "Ukraine", "United Kingdom", "Uruguay",
    "Zambia", "Zimbabwe"
    ]

for country in countries:
    url = f"https://restcountries.com/v3.1/name/{country}"
    response = requests.get(url).json()

    try:
        capital = response[0]["capital"][0]
        lat, lon = response[0]["capitalInfo"]["latlng"]
        print(f"Country: {country} | Capital: {capital} | Coordinates: {lat}, {lon}")
    except (KeyError, IndexError):
        print(f"Country: {country} | Capital information not available")

Country: Albania | Capital: Tirana | Coordinates: 41.32, 19.82
Country: Algeria | Capital: Algiers | Coordinates: 36.75, 3.05
Country: Angola | Capital: Luanda | Coordinates: -8.83, 13.22
Country: Argentina | Capital: Buenos Aires | Coordinates: -34.58, -58.67
Country: Armenia | Capital: Yerevan | Coordinates: 40.17, 44.5
Country: Australia | Capital: Canberra | Coordinates: -35.27, 149.13
Country: Austria | Capital: Vienna | Coordinates: 48.2, 16.37
Country: Azerbaijan | Capital: Baku | Coordinates: 40.38, 49.87
Country: Bahamas | Capital: Nassau | Coordinates: 25.08, -77.35
Country: Bahrain | Capital: Manama | Coordinates: 26.23, 50.57
Country: Bangladesh | Capital: Dhaka | Coordinates: 23.72, 90.4
Country: Belarus | Capital: Minsk | Coordinates: 53.9, 27.57
Country: Belgium | Capital: Brussels | Coordinates: 50.83, 4.33
Country: Botswana | Capital: Gaborone | Coordinates: -24.63, 25.9
Country: Brazil | Capital: Brasília | Coordinates: -15.79, -47.88
Country: Bulgaria | Capital: Sofi

In [55]:
import pandas as pd

# CSV faylını oxumaq
data = pd.read_csv('/content/drive/MyDrive/raw_data (1).csv')

# DataFrame-i yoxlamaq
print(data.head())


   Unnamed: 0     Area         Item  Year  hg/ha_yield  \
0           0  Albania        Maize  1990        36613   
1           1  Albania     Potatoes  1990        66667   
2           2  Albania  Rice, paddy  1990        23333   
3           3  Albania      Sorghum  1990        12500   
4           4  Albania     Soybeans  1990         7000   

   average_rain_fall_mm_per_year  pesticides_tonnes  
0                           1485              121.0  
1                           1485              121.0  
2                           1485              121.0  
3                           1485              121.0  
4                           1485              121.0  


In [56]:
data.sample(7)

,Unnamed: 0,Area,Item,Year,hg/ha_yield,average_rain_fall_mm_per_year,pesticides_tonnes
12307,12307,India,Potatoes,2000,186443,1083,44957.52
14537,14537,India,Wheat,2013,31538,1083,45620.00
685,685,Argentina,Potatoes,2010,289849,591,105915.04
777,777,Armenia,Wheat,2002,23877,562,88.77
14033,14033,India,Cassava,2011,364770,1083,55540.00
14867,14867,Indonesia,Soybeans,1998,11923,2702,1597.00
12845,12845,India,Potatoes,2004,188091,1083,35113.00


In [57]:
import requests
import pandas as pd

# Paytaxtların koordinatları və adları
capitals = [
    {'name': 'Baku', 'lat': 40.4093, 'lon': 49.8671},
    # Digər paytaxtlar burada əlavə oluna bilər
]

# API üçün əsas URL və açar
api_key = 'YOUR_API_KEY'
base_url = 'https://api.open-meteo.com/v1/historical'

# Nəticələri saxlamaq üçün boş bir siyahı
weather_data = []

# Hər bir paytaxt üçün dövr edirik
for capital in capitals:
    lat = capital['lat']
    lon = capital['lon']
    name = capital['name']

    # API sorğusu göndəririk
    params = {
        'latitude': lat,
        'longitude': lon,
        'start_date': '2025-11-24',
        'end_date': '2023-12-25',
        'hourly': 'temperature_2m,precipitation',
        'timezone': 'auto',
        'apikey': api_key
    }

    response = requests.get(base_url, params=params)

    if response.status_code == 200:
        data = response.json()
        data['city'] = name
        weather_data.append(data)
    else:
        print(f"Failed to get data for {name}")

# Nəticələri DataFrame-ə çeviririk
weather_df = pd.DataFrame(weather_data)

# Nəticəni çap edirik
print(weather_df.head())


Failed to get data for Baku
Empty DataFrame
Columns: []
Index: []


In [61]:
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import time

# Dataset yükləndi
df = pd.read_csv('/content/drive/MyDrive/raw_data (1).csv')

# Unikal ölkələri tap
countries = df['Area'].unique()

# Nominatim obyektini yarat
geolocator = Nominatim(user_agent="crop_prediction_app_v2", timeout=10)

# Saniyədə maksimum 1 sorğu göndərmək üçün RateLimiter
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1.5)

country_coords = {}

print(f"{len(countries)} ölkə üçün koordinatlar tapılır...")

for country in countries:
    try:
        # Sorğu göndər
        location = geocode(country)
        if location:
            country_coords[country] = (location.latitude, location.longitude)
            print(f"Tapıldı: {country} -> {location.latitude}, {location.longitude}")
        else:
            country_coords[country] = (None, None)
            print(f"Tapılmadı: {country}")
    except Exception as e:
        print(f"Xəta baş verdi ({country}): {e}")
        country_coords[country] = (None, None)
        time.sleep(2) # Xəta olduqda fasile

# Koordinatlar datasetə əlavə olunur
df['lat'] = df['Area'].map(lambda x: country_coords.get(x, (None, None))[0])
df['lon'] = df['Area'].map(lambda x: country_coords.get(x, (None, None))[1])

print("Əməliyyat tamamlandı!")

101 ölkə üçün koordinatlar tapılır...
Tapıldı: Albania -> 41.000028, 19.9999619
Tapıldı: Algeria -> 28.0000272, 2.9999825
Tapıldı: Angola -> -11.8775768, 17.5691241
Tapıldı: Argentina -> -34.9964963, -64.9672817
Tapıldı: Armenia -> 40.7696272, 44.6736646
Tapıldı: Australia -> -24.7761086, 134.755
Tapıldı: Austria -> 47.59397, 14.12456
Tapıldı: Azerbaijan -> 40.3936294, 47.7872508
Tapıldı: Bahamas -> 24.7736546, -78.0000547
Tapıldı: Bahrain -> 26.1551249, 50.5344606
Tapıldı: Bangladesh -> 24.4769288, 90.2934413
Tapıldı: Belarus -> 53.4250605, 27.6971358
Tapıldı: Belgium -> 50.6402809, 4.6667145
Tapıldı: Botswana -> -23.1681782, 24.5928742
Tapıldı: Brazil -> -10.3333333, -53.2
Tapıldı: Bulgaria -> 42.6073975, 25.4856617
Tapıldı: Burkina Faso -> 12.0753083, -1.6880314
Tapıldı: Burundi -> -3.426449, 29.9324519
Tapıldı: Cameroon -> 4.6125522, 13.1535811
Tapıldı: Canada -> 61.0666922, -107.991707
Tapıldı: Central African Republic -> 7.0323598, 19.9981227
Tapıldı: Chile -> -31.7613365, -71.31

In [64]:
# Unikal Ölkə və İl cütlərini (hədəfləri) yaradırıq
weather_targets = df[['Area', 'Year', 'lat', 'lon']].drop_duplicates().reset_index(drop=True)

print(f"Hazırdır! {len(weather_targets)} fərqli nöqtə üçün hava məlumatı çəkiləcək.")

Hazırdır! 2250 fərqli nöqtə üçün hava məlumatı çəkiləcək.


In [65]:
import requests
def get_weather_data(lat, lon, year):
    if pd.isna(lat) or pd.isna(lon):
        return None, None, None

    # URL-i yoxla
    url = (f"https://archive-api.open-meteo.com/v1/archive?"
           f"latitude={float(lat)}&longitude={float(lon)}&"
           f"start_date={year}-01-01&end_date={year}-12-31&"
           f"daily=temperature_2m_mean,rain_sum,snowfall_sum&timezone=auto")

    try:
        response = requests.get(url)

        # Əgər 400 və ya 429 xətası alırıqsa, səbəbi çap edək
        if response.status_code != 200:
            print(f"Xəta: {row['Area']} ({year}) üçün Status Code: {response.status_code}")
            print(response.text) # API-dan gələn xəta mesajı
            return None, None, None

        data = response.json()

        if 'daily' not in data:
            print(f"Məlumat tapılmadı: {row['Area']} ({year})")
            return None, None, None

        daily = data['daily']

        # Boş siyahıları yoxlayaq
        avg_temp = round(sum(daily['temperature_2m_mean']) / len(daily['temperature_2m_mean']), 2) if daily.get('temperature_2m_mean') else None
        total_rain = round(sum(daily['rain_sum']), 2) if daily.get('rain_sum') else None
        total_snow = round(sum(daily['snowfall_sum']), 2) if daily.get('snowfall_sum') else None

        return avg_temp, total_rain, total_snow

    except Exception as e:
        print(f"Sistem xətası: {e}")
        return None, None, None



In [66]:
from google.colab import files # Kompyuterə yükləmək üçün

def get_weather_data(lat, lon, year):
    if pd.isna(lat) or pd.isna(lon):
        return None, None, None
    url = (f"https://archive-api.open-meteo.com/v1/archive?"
           f"latitude={lat}&longitude={lon}&"
           f"start_date={year}-01-01&end_date={year}-12-31&"
           f"daily=temperature_2m_mean,precipitation_sum,snowfall_sum&"
           f"timezone=auto")
    try:
        response = requests.get(url, timeout=10)
        if response.status_code == 200:
            data = response.json()
            daily = data.get('daily', {})
            # Ortalama Temperatur
            avg_t = round(sum(daily.get('temperature_2m_mean', [])) / len(daily.get('temperature_2m_mean', [])), 2) if daily.get('temperature_2m_mean') else None
            # Cəmi Yağıntı
            total_p = round(sum(daily.get('precipitation_sum', [])), 2) if daily.get('precipitation_sum') else None
            # Cəmi Qar
            total_s = round(sum(daily.get('snowfall_sum', [])), 2) if daily.get('snowfall_sum') else None
            return avg_t, total_p, total_s
        return None, None, None
    except:
        return None, None, None

start = 0
end = 500
current_batch = weather_targets.iloc[start:end]
batch_results = []

print(f"{start} ilə {end} arası məlumatlar çəkilir...")

for index, row in current_batch.iterrows():
    t, p, s = get_weather_data(row['lat'], row['lon'], row['Year'])
    batch_results.append({
        'Area': row['Area'], 'Year': row['Year'],
        'avg_temp': t, 'total_precip': p, 'total_snow': s
    })

    if index % 10 == 0:
        print(f"Sətir {index} tamamlandı: {row['Area']} ({row['Year']})")

    # 2.5 saniyə fasilə
    time.sleep(2.5)

# yadda saxla və yüklə
filename = f"weather_data_{start}_{end}.csv"
batch_df = pd.DataFrame(batch_results)
batch_df.to_csv(filename, index=False)
files.download(filename) # Faylı avtomatik kompyuterinə yükləyir

print(f"Tamamlandı! {filename} faylı kompyuterinə yükləndi.")

0 ilə 500 arası məlumatlar çəkilir...
Sətir 0 tamamlandı: Albania (1990)
Sətir 10 tamamlandı: Albania (2000)
Sətir 20 tamamlandı: Albania (2011)
Sətir 30 tamamlandı: Algeria (1997)
Sətir 40 tamamlandı: Algeria (2008)
Sətir 50 tamamlandı: Angola (1994)
Sətir 60 tamamlandı: Angola (2005)
Sətir 70 tamamlandı: Argentina (1991)
Sətir 80 tamamlandı: Argentina (2001)
Sətir 90 tamamlandı: Argentina (2012)
Sətir 100 tamamlandı: Armenia (2000)
Sətir 110 tamamlandı: Armenia (2011)
Sətir 120 tamamlandı: Australia (1997)
Sətir 130 tamamlandı: Australia (2008)
Sətir 140 tamamlandı: Austria (1994)
Sətir 150 tamamlandı: Austria (2005)
Sətir 160 tamamlandı: Azerbaijan (1993)
Sətir 170 tamamlandı: Azerbaijan (2004)
Sətir 180 tamamlandı: Bahamas (1992)
Sətir 190 tamamlandı: Bahamas (2002)
Sətir 200 tamamlandı: Bahamas (2013)
Sətir 210 tamamlandı: Bahrain (1999)
Sətir 220 tamamlandı: Bahrain (2010)
Sətir 230 tamamlandı: Bangladesh (1996)
Sətir 240 tamamlandı: Bangladesh (2007)
Sətir 250 tamamlandı: Belaru

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Tamamlandı! weather_data_0_500.csv faylı kompyuterinə yükləndi.


In [67]:
# Köhnə siyahını boşalt ki, yeni hissə üçün yer açılsın
batch_results = []

In [68]:
from google.colab import files # Kompyuterə yükləmək üçün

def get_weather_data(lat, lon, year):
    if pd.isna(lat) or pd.isna(lon):
        return None, None, None
    url = (f"https://archive-api.open-meteo.com/v1/archive?"
           f"latitude={lat}&longitude={lon}&"
           f"start_date={year}-01-01&end_date={year}-12-31&"
           f"daily=temperature_2m_mean,precipitation_sum,snowfall_sum&"
           f"timezone=auto")
    try:
        response = requests.get(url, timeout=10)
        if response.status_code == 200:
            data = response.json()
            daily = data.get('daily', {})
            # Ortalama Temperatur
            avg_t = round(sum(daily.get('temperature_2m_mean', [])) / len(daily.get('temperature_2m_mean', [])), 2) if daily.get('temperature_2m_mean') else None
            # Cəmi Yağıntı
            total_p = round(sum(daily.get('precipitation_sum', [])), 2) if daily.get('precipitation_sum') else None
            # Cəmi Qar
            total_s = round(sum(daily.get('snowfall_sum', [])), 2) if daily.get('snowfall_sum') else None
            return avg_t, total_p, total_s
        return None, None, None
    except:
        return None, None, None


start = 500
end = 1000
current_batch = weather_targets.iloc[start:end]
batch_results = []

print(f"{start} ilə {end} arası məlumatlar çəkilir...")

for index, row in current_batch.iterrows():
    t, p, s = get_weather_data(row['lat'], row['lon'], row['Year'])
    batch_results.append({
        'Area': row['Area'], 'Year': row['Year'],
        'avg_temp': t, 'total_precip': p, 'total_snow': s
    })

    if index % 10 == 0:
        print(f"Sətir {index} tamamlandı: {row['Area']} ({row['Year']})")

    #  2.5 saniyə fasilə
    time.sleep(2.5)

# yadda saxla və yüklə
filename = f"weather_data_{start}_{end}.csv"
batch_df = pd.DataFrame(batch_results)
batch_df.to_csv(filename, index=False)
files.download(filename) # Faylı avtomatik kompyuterinə yükləyir

print(f"Tamamlandı! {filename} faylı kompyuterinə yükləndi.")

500 ilə 1000 arası məlumatlar çəkilir...
Sətir 500 tamamlandı: Colombia (2002)
Sətir 510 tamamlandı: Colombia (2013)
Sətir 520 tamamlandı: Croatia (2001)
Sətir 530 tamamlandı: Croatia (2012)
Sətir 540 tamamlandı: Denmark (1994)
Sətir 550 tamamlandı: Denmark (2005)
Sətir 560 tamamlandı: Dominican Republic (1995)
Sətir 570 tamamlandı: Dominican Republic (2006)
Sətir 580 tamamlandı: Ecuador (1992)
Sətir 590 tamamlandı: Ecuador (2002)
Sətir 600 tamamlandı: Ecuador (2013)
Sətir 610 tamamlandı: Egypt (1999)
Sətir 620 tamamlandı: Egypt (2010)
Sətir 630 tamamlandı: El Salvador (1996)
Sətir 640 tamamlandı: El Salvador (2007)
Sətir 650 tamamlandı: Eritrea (1996)
Sətir 660 tamamlandı: Eritrea (2007)
Sətir 670 tamamlandı: Estonia (1995)
Sətir 680 tamamlandı: Estonia (2006)
Sətir 690 tamamlandı: Finland (1992)
Sətir 700 tamamlandı: Finland (2002)
Sətir 710 tamamlandı: Finland (2013)
Sətir 720 tamamlandı: France (1999)
Sətir 730 tamamlandı: France (2010)
Sətir 740 tamamlandı: Germany (1996)
Sətir 75

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Tamamlandı! weather_data_500_1000.csv faylı kompyuterinə yükləndi.


In [70]:
from google.colab import files # Kompyuterə yükləmək üçün

def get_weather_data(lat, lon, year):
    if pd.isna(lat) or pd.isna(lon):
        return None, None, None
    url = (f"https://archive-api.open-meteo.com/v1/archive?"
           f"latitude={lat}&longitude={lon}&"
           f"start_date={year}-01-01&end_date={year}-12-31&"
           f"daily=temperature_2m_mean,precipitation_sum,snowfall_sum&"
           f"timezone=auto")
    try:
        response = requests.get(url, timeout=10)
        if response.status_code == 200:
            data = response.json()
            daily = data.get('daily', {})
            # Ortalama Temperatur
            avg_t = round(sum(daily.get('temperature_2m_mean', [])) / len(daily.get('temperature_2m_mean', [])), 2) if daily.get('temperature_2m_mean') else None
            # Cəmi Yağıntı
            total_p = round(sum(daily.get('precipitation_sum', [])), 2) if daily.get('precipitation_sum') else None
            # Cəmi Qar
            total_s = round(sum(daily.get('snowfall_sum', [])), 2) if daily.get('snowfall_sum') else None
            return avg_t, total_p, total_s
        return None, None, None
    except:
        return None, None, None

# --- PARAMETRLƏR ---
start = 1000
end = 1500
current_batch = weather_targets.iloc[start:end]
batch_results = []

print(f"{start} ilə {end} arası məlumatlar çəkilir...")

for index, row in current_batch.iterrows():
    t, p, s = get_weather_data(row['lat'], row['lon'], row['Year'])
    batch_results.append({
        'Area': row['Area'], 'Year': row['Year'],
        'avg_temp': t, 'total_precip': p, 'total_snow': s
    })

    if index % 10 == 0:
        print(f"Sətir {index} tamamlandı: {row['Area']} ({row['Year']})")

    # 2.5 saniyə fasilə
    time.sleep(2.5)

# yadda saxla və yüklə
filename = f"weather_data_{start}_{end}.csv"
batch_df = pd.DataFrame(batch_results)
batch_df.to_csv(filename, index=False)
files.download(filename) # Faylı avtomatik kompyuterinə yükləyir

print(f"Tamamlandı! {filename} faylı kompyuterinə yükləndi.")

1000 ilə 1500 arası məlumatlar çəkilir...
Sətir 1000 tamamlandı: Iraq (2004)
Sətir 1010 tamamlandı: Ireland (1990)
Sətir 1020 tamamlandı: Ireland (2000)
Sətir 1030 tamamlandı: Ireland (2011)
Sətir 1040 tamamlandı: Italy (1997)
Sətir 1050 tamamlandı: Italy (2008)
Sətir 1060 tamamlandı: Jamaica (1994)
Sətir 1070 tamamlandı: Jamaica (2005)
Sətir 1080 tamamlandı: Japan (1991)
Sətir 1090 tamamlandı: Japan (2001)
Sətir 1100 tamamlandı: Japan (2012)
Sətir 1110 tamamlandı: Kazakhstan (2000)
Sətir 1120 tamamlandı: Kazakhstan (2011)
Sətir 1130 tamamlandı: Kenya (1997)
Sətir 1140 tamamlandı: Kenya (2008)
Sətir 1150 tamamlandı: Latvia (1996)
Sətir 1160 tamamlandı: Latvia (2007)
Sətir 1170 tamamlandı: Lebanon (1993)
Sətir 1180 tamamlandı: Lebanon (2004)
Sətir 1190 tamamlandı: Lesotho (1990)
Sətir 1200 tamamlandı: Lesotho (2000)
Sətir 1210 tamamlandı: Lesotho (2011)
Sətir 1220 tamamlandı: Libya (1997)
Sətir 1230 tamamlandı: Libya (2008)
Sətir 1240 tamamlandı: Lithuania (1996)
Sətir 1250 tamamlandı: 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Tamamlandı! weather_data_1000_1500.csv faylı kompyuterinə yükləndi.


In [71]:
batch_results = []

In [72]:
from google.colab import files # Kompyuterə yükləmək üçün

def get_weather_data(lat, lon, year):
    if pd.isna(lat) or pd.isna(lon):
        return None, None, None
    url = (f"https://archive-api.open-meteo.com/v1/archive?"
           f"latitude={lat}&longitude={lon}&"
           f"start_date={year}-01-01&end_date={year}-12-31&"
           f"daily=temperature_2m_mean,precipitation_sum,snowfall_sum&"
           f"timezone=auto")
    try:
        response = requests.get(url, timeout=10)
        if response.status_code == 200:
            data = response.json()
            daily = data.get('daily', {})
            # Ortalama Temperatur
            avg_t = round(sum(daily.get('temperature_2m_mean', [])) / len(daily.get('temperature_2m_mean', [])), 2) if daily.get('temperature_2m_mean') else None
            # Cəmi Yağıntı
            total_p = round(sum(daily.get('precipitation_sum', [])), 2) if daily.get('precipitation_sum') else None
            # Cəmi Qar
            total_s = round(sum(daily.get('snowfall_sum', [])), 2) if daily.get('snowfall_sum') else None
            return avg_t, total_p, total_s
        return None, None, None
    except:
        return None, None, None

start = 1500
end = 2000
current_batch = weather_targets.iloc[start:end]
batch_results = []

print(f"{start} ilə {end} arası məlumatlar çəkilir...")

for index, row in current_batch.iterrows():
    t, p, s = get_weather_data(row['lat'], row['lon'], row['Year'])
    batch_results.append({
        'Area': row['Area'], 'Year': row['Year'],
        'avg_temp': t, 'total_precip': p, 'total_snow': s
    })

    if index % 10 == 0:
        print(f"Sətir {index} tamamlandı: {row['Area']} ({row['Year']})")

    # 2.5 saniyə fasilə
    time.sleep(2.5)

# yadda saxla və yüklə
filename = f"weather_data_{start}_{end}.csv"
batch_df = pd.DataFrame(batch_results)
batch_df.to_csv(filename, index=False)
files.download(filename) # Faylı avtomatik kompyuterinə yükləyir

print(f"Tamamlandı! {filename} faylı kompüterinə yükləndi.")

1500 ilə 2000 arası məlumatlar çəkilir...
Sətir 1500 tamamlandı: Nepal (1995)
Sətir 1510 tamamlandı: Nepal (2006)
Sətir 1520 tamamlandı: Netherlands (1992)
Sətir 1530 tamamlandı: Netherlands (2002)
Sətir 1540 tamamlandı: Netherlands (2013)
Sətir 1550 tamamlandı: New Zealand (1999)
Sətir 1560 tamamlandı: New Zealand (2010)
Sətir 1570 tamamlandı: Nicaragua (1996)
Sətir 1580 tamamlandı: Nicaragua (2007)
Sətir 1590 tamamlandı: Niger (1993)
Sətir 1600 tamamlandı: Niger (2004)
Sətir 1610 tamamlandı: Norway (1990)
Sətir 1620 tamamlandı: Norway (2000)
Sətir 1630 tamamlandı: Norway (2011)
Sətir 1640 tamamlandı: Pakistan (1997)
Sətir 1650 tamamlandı: Pakistan (2008)
Sətir 1660 tamamlandı: Papua New Guinea (1994)
Sətir 1670 tamamlandı: Papua New Guinea (2005)
Sətir 1680 tamamlandı: Peru (1991)
Sətir 1690 tamamlandı: Peru (2001)
Sətir 1700 tamamlandı: Peru (2012)
Sətir 1710 tamamlandı: Poland (1998)
Sətir 1720 tamamlandı: Poland (2009)
Sətir 1730 tamamlandı: Portugal (1995)
Sətir 1740 tamamlandı: 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Tamamlandı! weather_data_1500_2000.csv faylı kompüterinə yükləndi.


In [74]:
batch_results = []

In [75]:
from google.colab import files # Kompyuterə yükləmək üçün

def get_weather_data(lat, lon, year):
    if pd.isna(lat) or pd.isna(lon):
        return None, None, None
    url = (f"https://archive-api.open-meteo.com/v1/archive?"
           f"latitude={lat}&longitude={lon}&"
           f"start_date={year}-01-01&end_date={year}-12-31&"
           f"daily=temperature_2m_mean,precipitation_sum,snowfall_sum&"
           f"timezone=auto")
    try:
        response = requests.get(url, timeout=10)
        if response.status_code == 200:
            data = response.json()
            daily = data.get('daily', {})
            # Ortalama Temperatur
            avg_t = round(sum(daily.get('temperature_2m_mean', [])) / len(daily.get('temperature_2m_mean', [])), 2) if daily.get('temperature_2m_mean') else None
            # Cəmi Yağıntı
            total_p = round(sum(daily.get('precipitation_sum', [])), 2) if daily.get('precipitation_sum') else None
            # Cəmi Qar
            total_s = round(sum(daily.get('snowfall_sum', [])), 2) if daily.get('snowfall_sum') else None
            return avg_t, total_p, total_s
        return None, None, None
    except:
        return None, None, None


start = 2000
end = 2250
current_batch = weather_targets.iloc[start:end]
batch_results = []

print(f"{start} ilə {end} arası məlumatlar çəkilir...")

for index, row in current_batch.iterrows():
    t, p, s = get_weather_data(row['lat'], row['lon'], row['Year'])
    batch_results.append({
        'Area': row['Area'], 'Year': row['Year'],
        'avg_temp': t, 'total_precip': p, 'total_snow': s
    })

    if index % 10 == 0:
        print(f"Sətir {index} tamamlandı: {row['Area']} ({row['Year']})")

    # 2.5 saniyə fasilə
    time.sleep(2.5)

# yadda saxla və yüklə
filename = f"weather_data_{start}_{end}.csv"
batch_df = pd.DataFrame(batch_results)
batch_df.to_csv(filename, index=False)
files.download(filename) # Faylı avtomatik kompyuterinə yükləyir

print(f"Tamamlandı! {filename} faylı kompyuterinə yükləndi.")

2000 ilə 2250 arası məlumatlar çəkilir...
Sətir 2000 tamamlandı: Sweden (2013)
Sətir 2010 tamamlandı: Switzerland (1999)
Sətir 2020 tamamlandı: Switzerland (2010)
Sətir 2030 tamamlandı: Tajikistan (1998)
Sətir 2040 tamamlandı: Tajikistan (2009)
Sətir 2050 tamamlandı: Thailand (1995)
Sətir 2060 tamamlandı: Thailand (2006)
Sətir 2070 tamamlandı: Tunisia (1992)
Sətir 2080 tamamlandı: Tunisia (2002)
Sətir 2090 tamamlandı: Tunisia (2013)
Sətir 2100 tamamlandı: Turkey (1999)
Sətir 2110 tamamlandı: Turkey (2010)
Sətir 2120 tamamlandı: Uganda (1996)
Sətir 2130 tamamlandı: Uganda (2007)
Sətir 2140 tamamlandı: Ukraine (1995)
Sətir 2150 tamamlandı: Ukraine (2006)
Sətir 2160 tamamlandı: United Kingdom (1992)
Sətir 2170 tamamlandı: United Kingdom (2002)
Sətir 2180 tamamlandı: United Kingdom (2013)
Sətir 2190 tamamlandı: Uruguay (1999)
Sətir 2200 tamamlandı: Uruguay (2010)
Sətir 2210 tamamlandı: Zambia (1996)
Sətir 2220 tamamlandı: Zambia (2007)
Sətir 2230 tamamlandı: Zimbabwe (1993)
Sətir 2240 tama

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Tamamlandı! weather_data_2000_2250.csv faylı kompyuterinə yükləndi.


In [76]:
import glob

# 1. Bütün hissə-hissə yüklənən faylları bir siyahıya yığırıq
all_files = glob.glob("weather_data_*.csv")
print(f"Tapılan fayl sayı: {len(all_files)}")

# 2. Onları bir cədvəldə birləşdiririk
combined_weather = pd.concat([pd.read_csv(f) for f in all_files]).drop_duplicates().reset_index(drop=True)

# 3. Hava məlumatlarını ana datasetin (df) üzərinə 'Area' və 'Year' sütunlarına görə join edirik
final_df = pd.merge(df, combined_weather, on=['Area', 'Year'], how='left')

# 4. Yoxlayaq: Boş qalan (NaN) hava məlumatı varmı?
missing_count = final_df['avg_temp'].isnull().sum()
print(f"Hava məlumatı tapılmayan sətir sayı: {missing_count}")

# 5. YEKUN faylı kompyuterinə yüklə
final_df.to_csv("crop_yield_prediction_with_weather.csv", index=False)
from google.colab import files
files.download("crop_yield_prediction_with_weather.csv")

print("Hər şey hazır! Artıq bütün dünya üzrə hava və məhsuldarlıq məlumatları bir fayldadır.")

Tapılan fayl sayı: 5
Hava məlumatı tapılmayan sətir sayı: 15144


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Hər şey hazır! Artıq bütün dünya üzrə hava və məhsuldarlıq məlumatları bir fayldadır.


In [78]:
final_df.sample(7)

,Unnamed: 0,Area,Item,Year,hg/ha_yield,average_rain_fall_mm_per_year,pesticides_tonnes,lat,lon,avg_temp,total_precip,total_snow
18143,18143,Madagascar,Soybeans,2000,6096,1513,130.46,-18.924960,46.441642,21.05,1331.2,0.00
7560,7560,Ecuador,Sweet potatoes,1993,36276,2274,1844.00,-1.339767,-79.366697,21.46,7043.3,0.00
12202,12202,India,Soybeans,1999,11380,1083,46195.00,22.351115,78.667743,NaN,NaN,NaN
9850,9850,Guinea,Sweet potatoes,2013,34607,1651,877.19,10.722623,-10.708359,26.61,1155.4,0.00
16498,16498,Japan,Soybeans,1997,17380,1668,79821.18,36.574844,139.239418,8.21,1273.4,40.46
5012,5012,Burkina Faso,Yams,1996,73799,748,39.16,12.075308,-1.688031,28.35,565.0,0.00
14325,14325,India,Soybeans,2012,13530,1083,52980.00,22.351115,78.667743,NaN,NaN,NaN


In [1]:
final_df.to_sql(
    name="crop_yield_prediction_with_weather",
    con=engine,
    if_exists="replace",
    index=False
)

print("✅ Data loaded from .env and written to MySQL")

NameError: name 'final_df' is not defined